In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
!pip install 'qiskit[all]' 'qiskit-machine-learning'
!pip uninstall -y qiskit qiskit-machine-learning
!pip install -U qiskit qiskit-machine-learning
!apt update && apt install -y libomp-dev
!pip install torchviz

Found existing installation: qiskit 1.3.2
Uninstalling qiskit-1.3.2:
  Successfully uninstalled qiskit-1.3.2
Found existing installation: qiskit-machine-learning 0.8.2
Uninstalling qiskit-machine-learning-0.8.2:
  Successfully uninstalled qiskit-machine-learning-0.8.2
  Using cached qiskit-1.3.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached qiskit_machine_learning-0.8.2-py3-none-any.whl.metadata (13 kB)
Using cached qiskit-1.3.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.8 MB)
Using cached qiskit_machine_learning-0.8.2-py3-none-any.whl (231 kB)


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
20 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import Estimator
from qiskit_machine_learning.connectors import TorchConnector

In [1]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp.from_list([("ZZZZ", 1)])  # Example


In [16]:
data_train = '/content/drive/MyDrive/BreastMRI/Breast_Cancer_PatientsMRI/train'
data_valid = '/content/drive/MyDrive/BreastMRI/Breast_Cancer_PatientsMRI/validation'

In [17]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [18]:
train_dataset = datasets.ImageFolder(root=data_train, transform=transform)
valid_dataset = datasets.ImageFolder(root=data_valid, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [19]:
num_qubits = 4
feature_map = ZZFeatureMap(num_qubits)
ansatz = RealAmplitudes(num_qubits, entanglement='full')

quantum_circuit = QuantumCircuit(num_qubits)
quantum_circuit.compose(feature_map, inplace=True)
quantum_circuit.compose(ansatz, inplace=True)

In [33]:
from qiskit.quantum_info import SparsePauliOp

class QuantumLayer(nn.Module):
    def __init__(self, quantum_circuit, num_qubits):
        super().__init__()
        self.num_qubits = num_qubits
        self.quantum_circuit = quantum_circuit
        self.estimator = Estimator()
        self.weights = nn.Parameter(torch.randn(self.num_qubits))
        self.observable = SparsePauliOp.from_list([("Z" * num_qubits, 1.0)])  # Correct observable format

    def forward(self, x):
        batch_size = x.shape[0]
        outputs = []

        for i in range(batch_size):
            input_values = x[i].detach().cpu().numpy()
            param_dict = {param: val for param, val in zip(self.quantum_circuit.parameters, input_values)}

            # Explicitly pass the observable
            job = self.estimator.run([self.quantum_circuit], [param_dict], observables=[self.observable])
            result = job.result().values[0]
            outputs.append(result)

        return torch.tensor(outputs, dtype=torch.float32).unsqueeze(1)


In [31]:
class HQCNN(nn.Module):
    def __init__(self):
        super(HQCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(16 * 16 * 16, 64)
        self.fc2 = nn.Linear(64, 1)
        self.q_layer = quantum_layer

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.q_layer(x)
        x = torch.sigmoid(self.fc2(x))
        return x

model = HQCNN()

In [29]:
print(quantum_circuit.draw())


     ┌────────────────────────────────────┐»
q_0: ┤0                                   ├»
     │                                    │»
q_1: ┤1                                   ├»
     │  ZZFeatureMap(x[0],x[1],x[2],x[3]) │»
q_2: ┤2                                   ├»
     │                                    │»
q_3: ┤3                                   ├»
     └────────────────────────────────────┘»
«     ┌────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                       ├
«     │                                                                                                        │
«q_1: ┤1                                                                                                       ├
«     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7],θ[8],θ[9],θ[10],θ[11],θ[12],θ[13],θ[14],θ[15]) │
«q_2: ┤2                      

In [34]:
x_test = torch.rand(1, len(quantum_circuit.parameters))  # Adjust size
output = quantum_layer(x_test)
print(output)

QiskitError: "observable type not supported: <class 'qiskit.quantum_info.operators.symplectic.pauli_list.PauliList'>"

In [35]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for images, labels in train_loader:
        labels = labels.float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

QiskitError: "observable type not supported: <class 'qiskit.quantum_info.operators.symplectic.pauli_list.PauliList'>"